# 預售屋買賣資料
## 處理事項
- 由實價登錄網站抓取預售買賣資料，每月更新，更新時間同預售屋社區資料
- 資料清洗

In [1]:
import os
import re
import time
import datetime
from typing import Generator, List, Any, Literal
import pandas as pd
import requests
from tqdm import tqdm
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

c:\pylabs\housing-market-insights


In [3]:
from utils.configs import PRE_SALE_BASE_URL, PRE_SALE_URLS_FRAGMENTS, PRE_SALE_COLUMN_NAME
from utils.helper_function import build_complete_urls, combined_df, csv_extractor

## 從實價登錄網站取得預售屋買賣資料
- function
    - build_complete_url(網址組合)、fetch_data(向實價網址請求資料)、combined_df(合併全台預售屋資料)

In [4]:
# 組合預售屋「買賣」網址
urls = build_complete_urls(PRE_SALE_BASE_URL, PRE_SALE_URLS_FRAGMENTS)
for idx, (city, link) in enumerate(urls.items(), start=1):
    print(f"{idx}. {city} → {link}")

print(f"\n總共有 {len(urls)} 筆縣市資料")

1. 臺北市 → https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/98c5a4c3f2a084325cdcc53d9e548bb5?q=VTJGc2RHVmtYMStmaGRwU1NnQzdYdDFDdlNHejFUTzBNR2F1R25zb3JnOTFXYzljOG5DMmsrTmVqcTF1Y1pSb2czLzRWaUs1VmxNNzVFaE5uenhlQWhGbE9xcHlISEMrSnM1VzlTTk53TzE2cjhiRUx5RnFSNy9iQ2p4QW9aay9CSEcyZTdPVFErTUovMnpqSDZyTFJrRXMrWGZkUTQrQ01wdkE5NzB2clRhWmtML0Z3WnZGTUtKcXJMb3BVUDlQRW8vVkQ4TEl1c0ZkdGZTaGhVN0FjZHhEL3Mweit6bklGQ3B0OHY0YjNZL1l3SkZrcjNFZEhNcXNtcG1vaWtHa3Q1SXBsTGc5WnJVblpDZWsxeXh6SC9CL3J4OTJyWlZudmhDdjg3OGNONVN5N09Qd1RXTFg0allKS2lvb29obDZBSlgzNjV5cmJXZTEybHhKWHduRW9ERGkzTWRMamFjaEkyRFpPNU9CTUw0TkZiZ1dCbkVJN2hzZ3NtZ1pxWG5obWFGSnRVODZVKzFLeW9rT2FHalN1NzBVSy8wdDV0Q2Z3VlFUeDJVNzkrdzNHRERRYTFlUFpkdms3WGRXQW9pT1dOUldCMlArc2cwRDdOZGZrRS9LRHQ1STA5K3ZqV0JGaXZqeGVFZzhFeDZTSVRuZGg4NUprcGVjNllIYlFpMUR5c1Y4cENrUEhKUUJvSG0vd2RKK3NmRTc4R2JNV3FOLzJmd2xZV240dG5jNHRtQ3Y0RE1xQlFCZ283YmphTVBmOFZsTGpYTkNwUjIvMjdFN05XSit0RStZZ3VoVElnTXVxd0NqdWh2a1pGVHl2eTdLL0pWSUxQbUVTQ01ZRHdvakdUc3pFcEtFKy9pM0QvaGY5MVdjdVFUMFVTWkduOW5vUWNJTEY3bjI3d

In [5]:
# 依上述網址向實價登錄網站請求資料
raw_ps_transaction_df = combined_df(urls, "20250801")

開始處理各縣市資料：
處理 臺北市 中... 找到 20792 筆資料
處理 新北市 中... 找到 75765 筆資料
處理 桃園市 中... 找到 85453 筆資料
處理 新竹市 中... 找到 7284 筆資料
處理 新竹縣 中... 找到 19258 筆資料
處理 臺中市 中... 找到 83984 筆資料
處理 臺南市 中... 找到 36278 筆資料
處理 高雄市 中... 找到 44980 筆資料
處理 基隆市 中... 找到 4411 筆資料
處理 嘉義市 中... 找到 2821 筆資料
處理 宜蘭縣 中... 找到 9715 筆資料
處理 苗栗縣 中... 找到 12216 筆資料
處理 彰化縣 中... 找到 10611 筆資料
處理 南投縣 中... 找到 3571 筆資料
處理 雲林縣 中... 找到 7520 筆資料
處理 嘉義縣 中... 找到 5750 筆資料
處理 屏東縣 中... 找到 3928 筆資料
處理 臺東縣 中... 找到 912 筆資料
處理 花蓮縣 中... 找到 2364 筆資料

合併後資料總筆數: 437613 筆


In [6]:
# 儲存原始資料
output_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_transaction\raw"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "raw_ps_transaction_20250801.csv"
out_path = os.path.join(output_dir,  csv_fn)
raw_ps_transaction_df.to_csv(out_path, index=False, encoding='utf-8-sig')

## 載入原始資料並進行後續資料清洗
- function：csv_extractor(csv載入)

In [8]:
input_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\ps_transaction\raw"
csv_fn = "raw_ps_transaction_20250801.csv"
input_path = os.path.join(input_dir,  csv_fn)

In [9]:
# 載入原始預售屋社區資料
extracted =csv_extractor(input_path)
print(f" 逐筆交易資料載入成功: {extracted.shape}  記憶體使用: {extracted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
extracted

total_rows: 444061


Extracting rows:  99%|█████████▊| 437613/444061 [00:03<00:00, 114426.35it/s]



Extracting finished
 逐筆交易資料載入成功: (437613, 43)  記憶體使用: 906.73 MB


,city_name,AA11,AA12,a,b,bn,bs,bu,cinfo,city,...,s,sq,t,town,tp,tunit,type,unit,v,input_time
0,臺北市,住,NaN,北投區文林北路與文承路交叉口,住宅大樓(11層含以上有電梯),麟 璞真,62.04%,C-10F號,NaN,A,...,50.82,RUyjLmOIihYUmJ955O2ckodQrSsiHSFhUo3zIxvO3yA=,房地(土地+建物)+車位,A16,"62,820,000",1,Sale,2,3房2廳2衛,20250801
1,臺北市,住,NaN,北投區文林北路與文承路交叉口,住宅大樓(11層含以上有電梯),麟 璞真,49.22%,C-13F號,NaN,A,...,64.06,ZDJ+OsII2qvlVMcV6nc+9odQrSsiHSFhUo3zIxvO3yA=,房地(土地+建物)+車位,A16,"69,120,000",1,Sale,2,3房2廳2衛,20250801
2,臺北市,住,NaN,北投區文林北路與文承路交叉口,住宅大樓(11層含以上有電梯),麟 璞真,43.27%,A-11F號,NaN,A,...,45.08,MEQf4uz9JT1ZN+jIDh461YdQrSsiHSFhUo3zIxvO3yA=,房地(土地+建物)+車位,A16,"43,420,000",1,Sale,2,2房2廳2衛,20250801
3,臺北市,住,NaN,北投區文林北路與文承路交叉口,住宅大樓(11層含以上有電梯),麟 璞真,49.22%,C-12F號,NaN,A,...,64.06,qnKowQn/Hcv3XID6JIqDpYdQrSsiHSFhUo3zIxvO3yA=,房地(土地+建物)+車位,A16,"68,520,000",1,Sale,2,3房2廳2衛,20250801
4,臺北市,住,NaN,北投區文林北路與文承路交叉口,住宅大樓(11層含以上有電梯),麟 璞真,44.07%,A-13F號,NaN,A,...,44.25,gTYVJV6qxoi8MWJdfjvNeIdQrSsiHSFhUo3zIxvO3yA=,房地(土地+建物)+車位,A16,"44,780,000",1,Sale,2,2房2廳2衛,20250801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437608,花蓮縣,住,NaN,花蓮市富裕一街,華廈(10層含以下有電梯),璽鈺亞崴,45.71%,A棟8D號,NaN,U,...,31.41,CRDKfh840KKUVZemhEikxf2Sl90QGhM45/Zm91XlrLI=,房地(土地+建物)+車位,U01,"5,830,000",1,Sale,2,2房1廳1衛,20250801
437609,花蓮縣,住,NaN,花蓮市富裕一街,華廈(10層含以下有電梯),璽鈺亞崴,45.56%,A棟7B號,NaN,U,...,35.5,4ueJuogUzHEXgOPKTmjrCP2Sl90QGhM45/Zm91XlrLI=,房地(土地+建物)+車位,U01,"8,000,000",1,Sale,2,2房2廳2衛,20250801
437610,花蓮縣,住,NaN,吉安鄉昌隆三街,華廈(10層含以下有電梯),欣悅敦峰,55.49%,B棟03號,NaN,U,...,21.91,oz+QPLRbec5UxIWmV++AF/2Sl90QGhM45/Zm91XlrLI=,房地(土地+建物),U05,"5,200,000",1,Sale,2,1房2廳1衛,20250801
437611,花蓮縣,住,NaN,吉安鄉昌隆三街,華廈(10層含以下有電梯),欣悅敦峰,59.65%,C棟01號,1130301全部解約,U,...,26.84,Sl9Ph4O/QR8/ojh9KdHxpv2Sl90QGhM45/Zm91XlrLI=,房地(土地+建物),U05,"6,200,000",1,Sale,2,2房2廳1衛,20250801
